In [ ]:
#######################################################
# Script:
#    trainDecisionTree.py
# Usage:
#    python trainDecisionTree.py
# Description:
#    Build the prediction model based on training data
# Authors:
#    Jackie Chu,   cchu@salesforce.com
#    Jasmin Nakic, jnakic@salesforce.com
#######################################################

import numpy as np
from sklearn import tree
from sklearn.externals import joblib
import graphviz
import matplotlib.pyplot as plt

In [ ]:
# Enable or disable debug printing
debugFlag = True

# Feature list
perfCols = ["PageTime_ms","TotalServerTime_ms","TotalBrowserTime_ms","Action_count","Api_count","Db_count","DbTime_ms","Xhr_count"]

# GraphViz metadata
graphCols = ("dummy","PageTime_ms","TotalServerTime_ms","TotalBrowserTime_ms","Action_count","Api_count","Db_count","DbTime_ms","Xhr_count")
targetVals = ("Error","Invalid","Regression","Success")

# Model options
maxDepth = 3
minSamplesLeaf = 0


In [ ]:
# Add a set of columns to the destination
def addColumns(dest, src, colNames):
    # Initialize temporary array
    tmpArr = np.empty(src.shape[0])
    cols = 0
    # Copy column content
    for name in colNames:
        if cols == 0: # first column
            tmpArr = np.copy(src[name])
            tmpArr = np.reshape(tmpArr,(-1,1))
        else:
            tmpCol = np.copy(src[name])
            tmpCol = np.reshape(tmpCol,(-1,1))
            tmpArr = np.append(tmpArr,tmpCol,1)
        cols = cols + 1
    return np.append(dest,tmpArr,1)
#end addColumns

In [ ]:
# Generate Decision Tree model
def genModel(data,colList,modelName):
    # Prepare the data for the model
    X = np.zeros(data.shape[0])
    X = np.reshape(X,(-1,1))
    X = addColumns(X,data,colList)
    if debugFlag:
        print("X 0: ", X[0:5])
    Y = np.copy(data["Status"])
    if debugFlag:
        print("Y 0: ", Y[0:5])

    # Build the model based on training data
    model = tree.DecisionTreeClassifier()
    if maxDepth > 0 :
        model.set_params(max_depth=maxDepth)
    if minSamplesLeaf > 0:
        model.set_params(min_samples_leaf=minSamplesLeaf)
    
    print(model.fit(X, Y))

    print("NAMES: ", data.dtype.names)
    print("MAX_DEPTH: ", model.tree_.max_depth)
    print("NODE_COUNT: ", model.tree_.node_count)
    print("SCORE values: ", model.score(X,Y))
    
    # Print the feature ranking
    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1]
    print("Feature Importances ranking:")
    for f in range(X.shape[1]):
        print("\t%d. feature %d - %s (%f)" % (f + 1, indices[f], colList[indices[f]-1], importances[indices[f]]))

    # Visualize the decision tree model
    dot_data = tree.export_graphviz(model, out_file=None, 
                         feature_names=graphCols,
                         class_names=targetVals,
                         filled=True, rounded=True,  
                         special_characters=True,impurity=False) 
    
    # Directly display the graph
    graphviz.Source(dot_data).view()
    #graph = graphviz.Source(dot_data) 
    #graph.render(modelName)
    
    # Write the model to the file
    modelFileName = modelName+".model"
    joblib.dump(model,modelFileName)
    
    # use model to generate prediction for training data
    P = model.predict(X)
    if debugFlag:
        print("Prediction (for training data) 0-5: ", P[0:5])

    return P
#end genModel


In [ ]:
# Write training results to a file
def writeResult(output,data,p):
    result = np.array(
       np.empty(data.shape[0]),
       dtype=[
           ("PageTime_ms",int),
           ("TotalServerTime_ms",int),
           ("TotalBrowserTime_ms",int),
           ("Action_count",int),
           ("Api_count",int),
           ("Db_count",int),
           ("DbTime_ms",int),
           ("Xhr_count",int),
           ("Status","|U20"),
           ("PREDICTION","|U20")
        ]
    )
    result["PageTime_ms"]     = data["PageTime_ms"]
    result["TotalServerTime_ms"]     = data["TotalServerTime_ms"]
    result["TotalBrowserTime_ms"]     = data["TotalBrowserTime_ms"]
    result["Action_count"]     = data["Action_count"]
    result["Api_count"]    = data["Api_count"]
    result["Db_count"]    = data["Db_count"]
    result["DbTime_ms"]    = data["DbTime_ms"]
    result["Xhr_count"]    = data["Xhr_count"]
    result["Status"] = data["Status"]
    result["PREDICTION"] = p
    hdr = "PageTime_ms,TotalServerTime_ms,TotalBrowserTime_ms,Action_count,Api_count,Db_count,DbTime_ms,Xhr_count,Status,PREDICTION"
    np.savetxt(output,result,fmt="%s",delimiter=",",header=hdr,comments="")
    
    if debugFlag:
        print(hdr)
        print("Result 0-5: ", result[0:5])
#end writeResult

In [ ]:
# Start
inputFileName = "PerfRun_TrainingData.csv"
outputFileName = "PerfRun_TrainingResult.txt"
modelName = "PerfRun"

# All input columns - data types are strings, float and int
trainData = np.genfromtxt(
    inputFileName,
    delimiter=',',
    names=True,
    dtype=("|U20",int,int,int,int,int,int,int,int)
)
if debugFlag:
    print("Training Data : ", trainData[0:5])

In [ ]:
# Generate Decision Tree Model
P = genModel(trainData,perfCols,modelName)

In [ ]:
# Write result to file PerfRun_TrainingResult.txt
writeResult(outputFileName,trainData,P)